# Initialization

In [6]:
import vertexai
from vertexai.preview import reasoning_engines

PROJECT_ID = "hello-world-418507"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://2025-wy-agentspace"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [7]:
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent

def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (41 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}


root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash-exp",
    description=(
        "Agent to answer questions about the time and weather in a city."
    ),
    instruction=(
        "I can answer your questions about the time and weather in a city."
    ),
    tools=[get_weather, get_current_time],
)

In [11]:

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

## Local test

In [12]:
session = app.create_session(user_id="u_123")
session

Session(id='74078c23-44a6-4e43-bcb2-08e368afe4e3', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1747268455.5817358)

In [13]:
app.list_sessions(user_id="u_123")


ListSessionsResponse(sessions=[Session(id='74078c23-44a6-4e43-bcb2-08e368afe4e3', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1747268455.5817358)])

In [14]:
session = app.get_session(user_id="u_123", session_id=session.id)
session

Session(id='74078c23-44a6-4e43-bcb2-08e368afe4e3', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1747268455.5817358)

In [15]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the weather in london",
):
    print("*"*100)
    print(event)

****************************************************************************************************
{'content': {'parts': [{'function_call': {'id': 'adk-73324c11-3c4b-4918-85c2-ac3a8283597c', 'args': {'city': 'london'}, 'name': 'get_weather'}}], 'role': 'model'}, 'invocation_id': 'e-205e830c-eb54-4a22-8743-3bbb06849e71', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'j6E3Dat9', 'timestamp': 1747268457.488429}
****************************************************************************************************
{'content': {'parts': [{'function_response': {'id': 'adk-73324c11-3c4b-4918-85c2-ac3a8283597c', 'name': 'get_weather', 'response': {'status': 'error', 'error_message': "Weather information for 'london' is not available."}}}], 'role': 'user'}, 'invocation_id': 'e-205e830c-eb54-4a22-8743-3bbb06849e71', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_de

In [16]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the weather in new york",
):
    print("*"*100)
    print(event)

I0000 00:00:1747268465.715157 2355977 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


****************************************************************************************************
{'content': {'parts': [{'function_call': {'id': 'adk-94f261ab-7795-4473-b162-d6c303a46f70', 'args': {'city': 'new york'}, 'name': 'get_weather'}}], 'role': 'model'}, 'invocation_id': 'e-89dea7a7-7d11-4cc0-954d-7e19b776024c', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': '0m2srDq4', 'timestamp': 1747268465.696202}
****************************************************************************************************
{'content': {'parts': [{'function_response': {'id': 'adk-94f261ab-7795-4473-b162-d6c303a46f70', 'name': 'get_weather', 'response': {'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25 degrees Celsius (41 degrees Fahrenheit).'}}}], 'role': 'user'}, 'invocation_id': 'e-89dea7a7-7d11-4cc0-954d-7e19b776024c', 'author': 'weather_time_agent'

## Deploy

In [ ]:
from vertexai import agent_engines

# remote_app = agent_engines.create(
#     agent_engine=root_agent,
#     requirements=[
#         "google-cloud-aiplatform[adk,agent_engines]"   
#     ]
# )

## Test in new session

- Pre-deployed agent engine id: 3326620808347910144

In [15]:
live_app = vertexai.agent_engines.get('projects/671247654914/locations/us-central1/reasoningEngines/3326620808347910144')
session = live_app.create_session(user_id="u_123")
session

{'events': [],
 'user_id': 'u_123',
 'state': {},
 'id': '2228908531350765568',
 'app_name': '3326620808347910144',
 'last_update_time': 1747268934.567427}

In [ ]:
# Check available operations

# live_app.operation_schemas()

In [12]:
live_app.list_sessions(user_id="u_123")

{'sessions': [{'events': [],
   'last_update_time': 1747268875.576592,
   'state': {},
   'id': '589598266987905024',
   'app_name': '5830411094933372928',
   'user_id': 'u_123'},
  {'events': [],
   'last_update_time': 1747267742.264844,
   'state': {},
   'id': '8569976806688423936',
   'app_name': '5830411094933372928',
   'user_id': 'u_123'},
  {'events': [],
   'last_update_time': 1747267398.378696,
   'state': {},
   'id': '1652447779047342080',
   'app_name': '5830411094933372928',
   'user_id': 'u_123'}]}

In [13]:
import json

request_json = json.dumps(
    {
        # "artifacts": [
        #     {
        #         "file_name": "test_file_name",
        #         "versions": [{"version": "v1", "data": "v1data"}],
        #     }
        # ],
        # "authorizations": {
        #     "test_user_id1": {"access_token": "test_access_token"},
        #     "test_user_id2": {"accessToken": "test-access-token"},
        # },
        "user_id": "u_123",
        "message": {
            "parts": [{"text": "whats the weather in london?"}],
            "role": "user",
        },
    }
)
events = list(live_app.streaming_agent_run_with_events(request_json=request_json))
events

[{'events': [{'content': {'parts': [{'function_call': {'id': 'adk-0c7f9863-d95c-47b4-9377-cdd6855f5c4c',
        'args': {'city': 'london'},
        'name': 'get_weather'}}],
     'role': 'model'},
    'invocation_id': 'e-5d0f6895-86ba-4021-a990-e0bd74dee6ba',
    'author': 'weather_time_agent',
    'actions': {'state_delta': {},
     'artifact_delta': {},
     'requested_auth_configs': {}},
    'long_running_tool_ids': [],
    'id': '8fYx7sjf',
    'timestamp': 1747268897.677097}]},
 {'events': [{'content': {'parts': [{'function_response': {'id': 'adk-0c7f9863-d95c-47b4-9377-cdd6855f5c4c',
        'name': 'get_weather',
        'response': {'status': 'error',
         'error_message': "Weather information for 'london' is not available."}}}],
     'role': 'user'},
    'invocation_id': 'e-5d0f6895-86ba-4021-a990-e0bd74dee6ba',
    'author': 'weather_time_agent',
    'actions': {'state_delta': {},
     'artifact_delta': {},
     'requested_auth_configs': {}},
    'id': 'sn0DmCuT',
    't

### This method is obsolete

In [14]:
# for event in live_app.stream_query(
#     user_id="u_123",
#     session_id=session["id"],
#     message="whats the weather in london",
# ):
#     print("*"*100)
#     print(event)

## Logging

```bash
resource.type="aiplatform.googleapis.com/ReasoningEngine"
resource.labels.location="us-central1"
resource.labels.reasoning_engine_id="2590282269272834048"
```

In [11]:
from google.cloud import logging
from datetime import datetime, timedelta
import pytz

def get_recent_logs(engine_id, minutes=5):
    """
    Retrieve logs from a specific Reasoning Engine for the last specified minutes.
    
    Args:
        engine_id (str): The ID of the Reasoning Engine
        minutes (int): Number of minutes to look back for logs
    
    Returns:
        List of log entries
    """
    # Initialize the logging client
    client = logging.Client()
    
    # Calculate the time range in Pacific Time
    vegas_tz = pytz.timezone('America/Los_Angeles')
    end_time = datetime.now(vegas_tz)
    start_time = end_time - timedelta(minutes=minutes)
    
    # Format the start timestamp for the query
    start_time_str = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    # Construct the filter string
    filter_str = (
        'resource.type="aiplatform.googleapis.com/ReasoningEngine" '
        'resource.labels.location="us-central1" '
        f'resource.labels.reasoning_engine_id="{engine_id}" '
        f'timestamp >= "{start_time_str}"'
    )
    
    # Create an iterator for the logs and convert to list
    entries = list(client.list_entries(filter_=filter_str))
    
    print(f"Found {len(entries)} log entries in the last {minutes} minutes")
    
    # Print just the payload for each log entry
    for i, entry in enumerate(entries, 1):
        print(f"\n--- Log Entry {i} ---")
        print(f"Payload: {entry.payload}")



In [12]:
# Get logs for the last 15 minutes
get_recent_logs(engine_id="3326620808347910144", minutes=5)

Found 523 log entries in the last 5 minutes

--- Log Entry 1 ---
Payload: INFO:     Waiting for child process [54]

--- Log Entry 2 ---
Payload: INFO:     Child process [54] died

--- Log Entry 3 ---
Payload: INFO:     Waiting for child process [65]

--- Log Entry 4 ---
Payload: INFO:     Child process [65] died

--- Log Entry 5 ---
Payload: INFO:     Waiting for child process [69]

--- Log Entry 6 ---
Payload: INFO:     Child process [69] died

--- Log Entry 7 ---
Payload: INFO:     Waiting for child process [71]

--- Log Entry 8 ---
Payload: INFO:     Child process [71] died

--- Log Entry 9 ---
Payload: 05/13/2025 02:16:57 AM: Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>

--- Log Entry 10 ---
Payload: 05/13/2025 02:16:57 AM: Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>

--- Log

## Agentspace Registration using UI

- Open your AI Application using GCP console, and choose your Agentspace app
- Go to "Configuration" on the left menu -> Assistant tab
- Look for "Agents" section
- Now fill in the Agent Engine configuration as follows

![config](assets/agentspace-adk-register.png)

## Agentspace Registration (Obsolete)

In [ ]:
%%bash
export PROJECT_ID="hello-world-418507" 
export PROJECT_NUMBER="671247654914" 
export REASONING_ENGINE="projects/671247654914/locations/us-central1/reasoningEngines/3326620808347910144" 
export AGENT_DISPLAY_NAME="Wei WeatherAgent"
export AGENT_DESCRIPTION="Agent to answer questions about the time and weather in a city."
export AGENT_ID="wei_weather_agent" 
export AS_APP="2025-abc-gds_1741922567491" 

curl -X PATCH -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "x-goog-user-project: ${PROJECT_ID}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant?updateMask=agent_configs -d '{
    "name": "projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant",
    "displayName": "Default Assistant",
    "agentConfigs": [{
      "displayName": "'"${AGENT_DISPLAY_NAME}"'",
      "vertexAiSdkAgentConnectionInfo": {
        "reasoningEngine": "'"${REASONING_ENGINE}"'"
      },
      "toolDescription": "'"${AGENT_DESCRIPTION}"'",
      "icon": {
        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/corporate_fare/default/24px.svg"
      },
      "id": "'"${AGENT_ID}"'"
    }]
  }'

## Delete Agent Engine

In [ ]:
remote_app.delete(force=True)